In [20]:
import torch
import numpy as np
import librosa
import os
import pandas as pd
import tensorflow as tf

import soundfile as sf

from scipy.io import wavfile
from IPython.display import Audio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

from sklearn.preprocessing import StandardScaler




In [21]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

In [22]:
def getFiles(data, labels, direct):
    pathList = os.listdir(direct)
    for l in range(len(pathList)):
        if ".wav" in pathList[l]:
            data.append(direct + '/' +  pathList[l])
            continue
        elif ".txt" in pathList[l]:
            labels.append(direct + '/' +  pathList[l])
        else:
            temp = direct + '/' +  pathList[l]
            getFiles(data, labels, temp)
    return(data,labels)

In [23]:
t_data = []
t_label = []
test_data = []
test_label = []

folderName = 'Test'

t_data, t_label = getFiles(t_data,t_label ,folderName)

In [24]:
def extract_features(files):
    # Sample rate is set to 22050 by default
    X, sample_rate = librosa.load(files, sr=16000)
    # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
    stft = np.abs(librosa.stft(X))
        # Computes a chromagram from a waveform or power spectrogram.
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    # Computes a mel-scaled spectrogram.
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    # Computes spectral contrast
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    # Computes the tonal centroid features (tonnetz)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
    return (mfccs, chroma, mel, contrast, tonnetz)

In [25]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.callbacks import EarlyStopping

In [26]:

# Build a simple dense model with early stopping and softmax for categorical classification, remember we have 30 classes
model = Sequential()
model.add(Dense(193, input_shape=(193,), activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(30, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')

AttributeError: module 'tensorflow.python.framework.ops' has no attribute '_TensorLike'

In [ ]:
for i in range(len(t_data)):
    file_name = t_data[i]
    data = wavfile.read(file_name)
    framerate = data[0]
    sounddata = data[1]
    time = np.arange(0,len(sounddata))/framerate
    input_audio1, _ = librosa.load(file_name, sr=16000)
    print(input_audio1)


    train_features = extract_features(file_name)

    features_train = []
    features_train.append(np.concatenate((
        train_features[0],
        train_features[1], 
        train_features[2], 
        train_features[3],
        train_features[4]), axis=0))
    input_audio2 = np.array(features_train)
    input_audio2 = torch.DoubleTensor(input_audio2)


    input_values = tokenizer(input_audio1, return_tensors="pt").input_values
    print(input_audio2)
    print(input_values)
    print(input_values[0][0])
    print(input_audio2)
    
    history = model.fit(X_train, y_train, batch_size=256, epochs=100, 
                        validation_data=(X_val, y_val),
                        callbacks=[early_stop])

    logits = model(input_audio2).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    print(transcription)

[0.09133911 0.09155273 0.09182739 ... 0.01727295 0.01834106 0.01950073]


C:\Users\alexp\AppData\Local\Temp/ipykernel_21828/2747416684.py:11: FutureWarning: Pass y=[0.09133911 0.09155273 0.09182739 ... 0.01727295 0.01834106 0.01950073] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


ValueError: 'pt2' is not a valid TensorType, please select one of ['pt', 'tf', 'np', 'jax']